## Column Transformer

We will demonstrate why this is needed by doing things in the hard way and then the easy way.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
# import a dummy dataset
df = pd.read_csv('covid_toy.csv')
df.head()

# age - numeric
# gender - nominal
# fever - numeric
# cough - ordinal
# city - nominal
# has_covid - nominal

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [11]:
df.isnull().sum()
# looks like fever has missing values

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)
# note that df.drop() will returna data frame which doesn't have 'has_covid' column --- all the inputs

In [16]:
x_train

,age,gender,fever,cough,city
79,48,Female,103.0,Mild,Kolkata
10,75,Female,NaN,Mild,Delhi
97,20,Female,101.0,Mild,Bangalore
63,10,Male,100.0,Mild,Bangalore
85,16,Female,103.0,Mild,Bangalore
...,...,...,...,...,...
15,70,Male,103.0,Strong,Kolkata
4,65,Female,101.0,Mild,Mumbai
19,42,Female,NaN,Strong,Bangalore
78,11,Male,100.0,Mild,Bangalore


### Hard Way

In [21]:
# adding simple imputer to fill the empty columns
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also on the test data
x_test_fever = si.fit_transform(x_test[['fever']])

# this will fill the null with mean of data
x_train_fever.shape

(80, 1)

In [28]:
# ordinal encoding --- on cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also on the test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [33]:
# one hot encoding --- gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
# will drop the first column and not create a sparse matrix
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])

# also doing it on test
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

x_train_gender_city.shape
# 1 column from gender and 3 from city(after dropping one)

(80, 4)

In [35]:
# get the numeric columns
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

# also on the test data
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

x_train_age.shape

(80, 1)

In [39]:
# now concatinate all the columns
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough), axis=1)

# also for test data
x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough), axis=1)

x_train_transformed.shape
# now we have all the encoded columns (7)

(80, 7)

### Easy Way

In [41]:
from sklearn.compose import ColumnTransformer

In [49]:
# create an object
# remainder='passthrough' means it will just leave the untouched columns as-is,
# remainder='drop' will keep only the columns which were operated on
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'),['gender', 'city'])
], remainder='passthrough')

# always specify ('name', Class from sklearn(with its parameters), ['Column Names from original dataset])

In [50]:
transformer.fit_transform(x_train).shape
# we got the same matrix as last time but way less coding

(80, 7)

In [52]:
# similarly for the test data
transformer.fit_transform(x_test).shape

(20, 7)